# Applied Project in Big Data on Industrial Dataset

## DATA COLLECTION TECHNIQUES
## Part I. Web scraping intro

### 1. Libraries

Let's start from very basic example, we wiil need [urllib](https://docs.python.org/3/library/urllib.html) library for  opening and reading URLs. We will also use [Beautiful Soup](https://beautiful-soup-4.readthedocs.io/en/latest/) Python library to parce HTML data.

In [ ]:
import os
import re
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup

### 2. Get text from HTML page

In [ ]:
URL_2_SCRAP = 'https://ai-jobs.net'

In [ ]:
request = Request(URL_2_SCRAP)
response = urlopen(request)
html = response.read()

In [ ]:
html

In [ ]:
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
soup

In [ ]:
soup.contents

In [ ]:
soup.text

In [ ]:
text = soup.text
for ch in ['\n', '\t', '\r']:
    text = text.replace(ch, ' ')

In [ ]:
' '.join(text.split())

### 3. Simple NLP analysis

In [ ]:
# lower text and leave only text without symbols
text = re.sub('[^а-яА-Яa-zA-Z]+', ' ', text).strip().lower()
text

In [ ]:
text_as_list = text.split()
text_as_list[:5]

In [ ]:
from collections import Counter

In [ ]:
Counter(text_as_list)

In [ ]:
freqs = dict(Counter(text_as_list))

In [ ]:
freqs

In [ ]:
freqs = dict(
    sorted(
        freqs.items(), 
        key=lambda item: item[1], 
        reverse=True
    )
)
freqs

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
freqs_bar = {k: v for k, v in freqs.items() if v >= 10}

In [ ]:
plt.figure(figsize=(16, 6))
plt.bar(*zip(*freqs_bar.items()))
plt.xticks(rotation='vertical')
plt.show()

### 4. Save HTML to disk as row data

In [ ]:
with open('ai_jobs_page.html', 'w') as file:
    file.write(html.decode())

In [ ]:
with open('ai_jobs_page.html', 'r') as file:
    text = file.read()

In [ ]:
text

### 5. More than text

In [ ]:
soup

In [ ]:
soup.find_all('img')

In [ ]:
soup.find_all('img', attrs={'alt': 'Oak Ridge National Laboratory logo'})

In [ ]:
soup.find_all('img', attrs={'alt': re.compile(r".*Gorovoy")})

In [ ]:
soup.find_all('img')[19]['src']

In [ ]:
URL_2_SCRAP_IMG = 'https://ai-jobs.net/' + soup.find_all('img')[19]['src']
print(URL_2_SCRAP_IMG)

In [ ]:
request = Request(URL_2_SCRAP_IMG)
response = urlopen(request)
img = response

In [ ]:
img

In [ ]:
from PIL import Image
import numpy as np

In [ ]:
plt.figure(figsize=(12, 8))
img = Image.open(img)
plt.imshow(np.array(img))
plt.show()

In [ ]:
img.save('logo.jpg')

### 6. Get data

In [ ]:
soup

In [ ]:
soup.find('a')

In [ ]:
soup.find_all('a')

In [ ]:
soup.find_all('a', class_="col list-group-item-action px-2 py-3")

In [ ]:
jobs_list = soup.find_all('a', class_="col list-group-item-action px-2 py-3")

In [ ]:
jobs_list[0]

In [ ]:
jobs_list[1]

In [ ]:
jobs_list[1]['href']

In [ ]:
jobs_list[1]['title']

In [ ]:
jobs_list[1].find_all('span')

In [ ]:
jobs_list[1].find_all('span')[0]['class']

In [ ]:
jobs_list[1].find('span', class_='d-block d-md-none text-break job-list-item-location')

In [ ]:
jobs_list[1].find_all('span', class_='badge badge-light badge-pill')

In [ ]:
[x.text for x in jobs_list[1].find_all('span', class_='badge badge-light badge-pill')]

In [ ]:
for job in jobs_list:
    print(job['title'])
    print(URL_2_SCRAP + job['href'])
    print(
        'location ->',
        job.find('span', class_='d-block d-md-none text-break job-list-item-location').text
    )
    print(
        'requirements ->',
        [x.text for x in job.find_all('span', class_='badge badge-light badge-pill')]
    )
    print('*' * 50)

In [ ]:
all_jobs = []
for job in jobs_list:
    job_dict = {}
    job_dict['description'] = job['title']
    job_dict['url'] = URL_2_SCRAP + job['href']
    location = job.find(
        'span', 
        class_='d-block d-md-none text-break job-list-item-location'
    )
    job_dict['location'] = location.text if location else ''
    time = job.find(
        'span', 
        class_='badge badge-secondary badge-pill my-md-1'
    )
    job_dict['time'] = time.text if time else ''
    position = job.find(
        'span', 
        class_='badge badge-info badge-pill my-md-1 d-md-none'
    )
    job_dict['position'] = position.text if position else ''
    level = job.find(
        'span', 
        class_='badge badge-info badge-pill my-md-1 d-none d-md-inline-block'
    )
    job_dict['level'] = level.text if level else ''
    salary_range = job.find(
        'span', 
        class_='badge badge-success badge-pill d-none d-md-inline-block'
    )
    job_dict['salary_range'] = salary_range.text if salary_range else ''
    salary = job.find(
        'span', 
        class_='badge badge-success badge-pill d-md-none'
    )
    job_dict['salary'] = salary.text if salary else ''
    job_dict['requirements'] = [
        x.text for x in job.find_all('span', class_='badge badge-light badge-pill')
    ]
    job_dict['offers'] = [
        x.text for x in job.find_all('span', class_='badge badge-success badge-pill')
    ]
    all_jobs.append(job_dict)
all_jobs

In [ ]:
import pandas as pd

In [ ]:
pd.DataFrame(all_jobs)

### 7. Get data by single item

In [ ]:
URL_2_SCRAP = 'https://ai-jobs.net/job/32218-data-science-content-intern-remote/'

In [ ]:
request = Request(URL_2_SCRAP)
response = urlopen(request)
html = response.read()

In [ ]:
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
soup

In [ ]:
soup.find('script', type="application/ld+json")

In [ ]:
import json

In [ ]:
text = soup.find('script', type="application/ld+json").text
print(text)

In [ ]:
data = json.loads(text)
data

In [ ]:
data['baseSalary']

In [ ]:
data['identifier']['value']

### 8. Saving data

In [ ]:
folder = 'ai_jobs_data'
os.makedirs(folder, exist_ok=True)

In [ ]:
for job in all_jobs[:3]:
    print(job['url'])

In [ ]:
import time
from tqdm.auto import tqdm

In [ ]:
for job in tqdm(all_jobs):
    request = Request(job['url'])
    response = urlopen(request)
    html = response.read()
    soup = BeautifulSoup(html, 'html.parser')
    
    text = soup.find('script', type="application/ld+json").text
    data = json.loads(text, strict=False)
   
    job_id =  data['identifier']['value']
    file_path = f'{folder}/{job_id}.json'
    with open(file_path, 'w') as file:
        json.dump(data, file)
    
    time.sleep(.7)